In [1]:
import surrogate_model
import argparse
import numpy as np
import pandas as pd
import src.dynamic_pipeline as dynamic_pipeline

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('./data/concrete.csv')
# df = dynamic_pipeline.preprocess_dynamic(df)

In [3]:
ret_dict = dynamic_pipeline.preprocess_dynamic(df)

제거된 컬럼이 없습니다.
cement: 비대칭 분포로 확인됨. IQR 방식 적용.
slag: 비대칭 분포로 확인됨. IQR 방식 적용.
ash: 비대칭 분포로 확인됨. IQR 방식 적용.
water: 비대칭 분포로 확인됨. IQR 방식 적용.
superplastic: 비대칭 분포로 확인됨. IQR 방식 적용.
coarseagg: 비대칭 분포로 확인됨. IQR 방식 적용.
fineagg: 비대칭 분포로 확인됨. IQR 방식 적용.
age: 비대칭 분포로 확인됨. IQR 방식 적용.
strength: 비대칭 분포로 확인됨. IQR 방식 적용.
cement: 값 범위 중요 → MinMaxScaler 적용
slag: 값 범위 중요 → MinMaxScaler 적용
ash: 값 범위 중요 → MinMaxScaler 적용
water: 값 범위 중요 → MinMaxScaler 적용
superplastic: 값 범위 중요 → MinMaxScaler 적용
coarseagg: 값 범위 중요 → MinMaxScaler 적용
fineagg: 값 범위 중요 → MinMaxScaler 적용
age: 비대칭 분포 → RobustScaler 적용
strength: 값 범위 중요 → MinMaxScaler 적용


In [4]:
ret_dict['processed_df'].to_csv('./data/temp.csv', index=False)

In [5]:
# args = argparse.Namespace(
#     target = ['strength'],
#     data_path='./data/temp.csv',
#     model='catboost',
#     prj_id = 40,
#     seed=40
# )
# surrogate_model.main(args, ret_dict['scalers'])

In [6]:
args = argparse.Namespace(
    target = ['strength'],
    data_path='./data/temp.csv',
    model='tabpfn',
    prj_id = 40,
    seed=40
)
surrogate_model.main(args, ret_dict['scalers'])

2025-02-03 09:57:32,436 - INFO - X_train.shape: (740, 8)
2025-02-03 09:57:32,437 - INFO - X_test.shape: (186, 8)
2025-02-03 09:57:32,438 - INFO - y_train.shape: (740, 1)
2025-02-03 09:57:32,438 - INFO - y_test.shape: (186, 1)
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(186, 1)
(186, 1)
       rmse       mae        r2
0  0.056403  0.033312  0.926249
(186, 1) (186, 1)
(186, 1)


In [7]:
import os

In [8]:
os.listdir('./prj/40/surrogate_model')

['importance.csv', 'eval.csv', 'surrogate_model.csv', 'model.cbm', 'model.pkl']

In [9]:
import search_model

In [10]:
controll_range = df.drop(columns=['strength']).columns.tolist()
controll_range = {col: (df[col].min(), df[col].max()) for col in controll_range}


In [11]:
controll_range

{'cement': (102.0, 540.0),
 'slag': (0.0, 359.4),
 'ash': (0.0, 200.1),
 'water': (121.8, 247.0),
 'superplastic': (0.0, 32.2),
 'coarseagg': (801.0, 1145.0),
 'fineagg': (594.0, 992.6),
 'age': (1, 365)}

In [12]:
print(df['strength'].min(),'~',df['strength'].max())

2.33 ~ 82.6


In [13]:
from src.datasets.data_loader import load_data

In [14]:
df_scaled = load_data(args.data_path)
X = df_scaled.drop(columns=args.target)
y = df_scaled[args.target]
x_col_list = X.columns.tolist()

X_train,y_train = X.to_numpy(),y.to_numpy()


In [15]:
scalers = ret_dict['scalers']

In [16]:
# user_req = scalers['strength'].transform(np.array(args.user_request_target[0]).reshape(-1,1))

In [17]:
ur = []
ur.append(np.array(40.0).reshape(-1,1)[0])
ur = np.array(ur)

In [18]:
ur

array([[40.]])

In [19]:
x,y = search_model.find_top_k_similar_with_user_request(ur,X_train,y_train,k=50)

In [20]:
x.shape
y.shape


(50, 1)

In [21]:
from src.datasets.data_loader import load_data

In [22]:
args = argparse.Namespace(
    model='catboost',
    search_model='k_means',
    data_path='./data/temp.csv',
    controll_name=['cement', 'slag', 'ash', 'water', 'superplastic', 'coarseagg', 'fineagg', 'age'],
    controll_range= controll_range,
    target=['strength'],
    importance={'cement': 1,
                'water': 2,
    },
    optimize={'cement': 'minimize',
                'water': 'minimize',
    },
    prj_id=40,
    seed=40,
    user_request_target=[40.0]
)

In [23]:
# %%time 
x_opt = search_model.main(args,ret_dict['scalers'])

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/utils/valida

x_min :  [ 0.          0.          0.         -0.05148515  0.          0.
 -0.03235294 -1.28571429]
x_max :  [ 1.          1.05057001  1.          1.18811881  1.45701357  1.
  1.14       16.04761905]


0it [00:00, ?it/s]WARNING clustering 200 points to 66 centroids: please provide at least 2574 training points
WARNING clustering 200 points to 66 centroids: please provide at least 2574 training points
WARNING clustering 200 points to 66 centroids: please provide at least 2574 training points
WARNING clustering 200 points to 66 centroids: please provide at least 2574 training points
WARNING clustering 200 points to 66 centroids: please provide at least 2574 training points
WARNING clustering 200 points to 66 centroids: please provide at least 2574 training points
WARNING clustering 200 points to 66 centroids: please provide at least 2574 training points
WARNING clustering 200 points to 66 centroids: please provide at least 2574 training points
WARNING clustering 200 points to 66 centroids: please provide at least 2574 training points
WARNING clustering 200 points to 66 centroids: please provide at least 2574 training points
WARNING clustering 200 points to 66 centroids: please provide 

search model 소요 시간: 49.5808초


In [32]:
np.stack(x_opt['pred_x'].to_numpy()).shape

(50, 8)

In [34]:
deap==1.4.2
faiss-gpu==1.7.2
lightgbm==4.5.0
optuna==4.2.0
pandas==2.2.3
scikit-learn==1.6.1


SyntaxError: invalid syntax (1565910012.py, line 1)